In [2]:
!pip install onnx==1.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 39.8 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

# Replace 'path_to_your_file.csv' with the actual path to your CSV file in Google Drive
data = pd.read_csv('/content/drive/My Drive/merged_data.csv')



In [6]:
unique_countries = data["Country"].unique()
country_to_code = {country: code for code, country in enumerate(unique_countries)}
country_to_code

def apply_code(row):
  row["Country Code"] = country_to_code[row["Country"]]
  return row

data = data.apply(apply_code, axis=1)

In [7]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split

In [8]:
x = data[["Country Code", "Electricity_day_price", 'Difficulty', 'HashRate', 'Power', 'Block Reward', 'Cost_2_months', 'Cost_3_months', 'Cost_4_months', 'Cost_5_months', 'Cost_6_months']].values

In [9]:
y = data[['BTC_needed_2_months', 'BTC_needed_3_months', 'BTC_needed_4_months', 'BTC_needed_5_months', 'BTC_needed_6_months']].values

In [10]:
x_tensor = torch.tensor(x, dtype=torch.float32)

In [11]:
y_tensor = torch.tensor(y, dtype=torch.float32)

In [12]:
dataset = TensorDataset(x_tensor, y_tensor)

In [13]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [14]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [20]:
class BitcoinPredictor(nn.Module):
    def __init__(self):
        super(BitcoinPredictor, self).__init__()

        # Input layer to first hidden layer
        self.fc1 = nn.Linear(11, 100)  # 6000
        self.relu1 = nn.ReLU()  # Activation function

        # First hidden layer to second hidden layer
        self.fc2 = nn.Linear(100, 200)  # 12000
        self.relu2 = nn.ReLU()  # Activation function

        # First hidden layer to second hidden layer
        self.fc3 = nn.Linear(200, 100)  # 5000
        self.relu3 = nn.ReLU()  # Activation function

        # First hidden layer to second hidden layer
        self.fc4 = nn.Linear(100, 7)  # 500
        self.relu4 = nn.ReLU()  # Activation function

        # Second hidden layer to output layer
        self.fc5 = nn.Linear(7, 5)  # Output size is 1 (predicted price)

    def forward(self, x):
        x = self.fc1(x)  # Pass through first layer
        x = self.relu1(x)  # Apply activation

        x = self.fc2(x)  # Pass through second layer
        x = self.relu2(x)  # Apply activation

        x = self.fc3(x)  # Pass through output layer
        x = self.relu3(x)  # Apply activation

        x = self.fc4(x)  # Pass through output layer
        x = self.relu4(x)  # Apply activation

        x = self.fc5(x)  # Pass through output layer
        return x

In [21]:
model = BitcoinPredictor()

criterion = nn.L1Loss()
# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000007)

num_epochs = 10

In [22]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Move data to GPU
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device)


In [23]:
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

In [24]:
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = to_device(X_batch, device), to_device(y_batch, device)
        # Forward pass
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Print the loss for every epoch
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print(f'Outputs: {outputs}')
        print(f'y_batch: {y_batch}')


Streaming output truncated to the last 5000 lines.
Epoch [10/10], Loss: 0.0228
Epoch [10/10], Loss: 0.0247
Epoch [10/10], Loss: 0.0358
Epoch [10/10], Loss: 0.0304
Epoch [10/10], Loss: 0.0332
Epoch [10/10], Loss: 0.0404
Epoch [10/10], Loss: 0.0367
Epoch [10/10], Loss: 0.0364
Epoch [10/10], Loss: 0.0310
Epoch [10/10], Loss: 0.0251
Epoch [10/10], Loss: 0.0402
Epoch [10/10], Loss: 0.0393
Epoch [10/10], Loss: 0.0198
Epoch [10/10], Loss: 0.0277
Epoch [10/10], Loss: 0.0267
Epoch [10/10], Loss: 0.0373
Epoch [10/10], Loss: 0.0417
Epoch [10/10], Loss: 0.0206
Epoch [10/10], Loss: 0.0404
Epoch [10/10], Loss: 0.0514
Epoch [10/10], Loss: 0.0274
Epoch [10/10], Loss: 0.0483
Epoch [10/10], Loss: 0.0361
Epoch [10/10], Loss: 0.0414
Epoch [10/10], Loss: 0.0247
Epoch [10/10], Loss: 0.0499
Epoch [10/10], Loss: 0.0344
Epoch [10/10], Loss: 0.0367
Epoch [10/10], Loss: 0.0401
Epoch [10/10], Loss: 0.0382
Epoch [10/10], Loss: 0.0281
Epoch [10/10], Loss: 0.0395
Epoch [10/10], Loss: 0.0432
Epoch [10/10], Loss: 0.03

In [25]:
model.eval()
with torch.no_grad():
    total_loss = 0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = to_device(X_batch, device), to_device(y_batch, device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        print(f'{outputs} - {y_batch}')
        total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    print(f'Average Test Loss: {average_loss:.4f}')

Streaming output truncated to the last 5000 lines.
        [0.0287, 0.0447, 0.0629, 0.0863, 0.1087]], device='cuda:0')
tensor([[0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712]], device='cuda:0') - tensor([[0.1187, 0.2183, 0.3344, 0.4106, 0.4654],
        [0.0221, 0.0287, 0.0342, 0.0383, 0.0419],
        [0.0209, 0.0309, 0.0416, 0.0553, 0.0709],
        [0.0208, 0.0281, 0.0340, 0.0413, 0.0499],
        [0.0119, 0.0148, 0.0181, 0.0206, 0.0235]], device='cuda:0')
tensor([[0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712],
        [0.0215, 0.0329, 0.0450, 0.0576, 0.0712]], device='cuda:0') - tensor([[0.0380, 0.0657, 0.0969, 0.1274, 0.1574],
        [0.0191, 0.0285

In [26]:
onnx_model_path = "model_v1.onnx"
input_tensor = to_device(x_tensor, device)
torch.onnx.export(model, input_tensor, onnx_model_path)